In [30]:
# import necessary packages
import numpy as np
import scipy as sci
import matplotlib as plt

In [31]:
class QuantumRegister:
    def __init__(self, N):
        # Initialize column vector
        self.N = N
        self.psi = 1 / np.sqrt(2**self.N) * np.zeros((2**N, 1), dtype=complex)
        # Set the amplitude of the first state
        self.psi[0] = 1
    
    def get_register(self):
        # Method to return the quantum state
        return self.psi
    
    def measure_state(self):
        probabilities = np.abs(self.psi)**2
        
        # select a basis state randomly according to the probabilities
        measurement = np.random.choice(np.arange(len(self.psi)), p=probabilities.ravel())
        return "| " + np.binary_repr(measurement, width = self.N) + " >"

    def ApplyGate(self,gate):
        self.psi = np.matmul(gate,self.psi)

In [38]:
class QuantumGates:
    
    @staticmethod
    def HadamardTransform(N):
        # define the initial Hadamard matrix
        hadamard_initial = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

        # number of iterations for the transformation
        num_iterations = N - 1

        # initialize the resulting tensor product matrix
        hadamard_result = hadamard_initial

        # perform tensor product iteratively
        for i in range(num_iterations):
            hadamard_result = np.kron(hadamard_initial, hadamard_result)

        return hadamard_result
    
    @staticmethod
    def ApplyHadamard(QuantReg):
        N = QuantReg.N
        HadamardGate = QuantumGates.HadamardTransform(N)
        QuantReg.ApplyGate(HadamardGate)


In [41]:
"""
TESTING BLOCK
"""

"""creates the quantum register """
quantum_Register1 = QuantumRegister(3)
print(quantum_Register1.get_register())
print(quantum_Register1.measure_state())

"""operating on the register using the gates"""
#Applying the Hadamard
QuantumGates.ApplyHadamard(quantum_Register1)
print(quantum_Register1.get_register())

#Applying the Oracle


[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
| 000 >
[[0.35355339+0.j]
 [0.35355339+0.j]
 [0.35355339+0.j]
 [0.35355339+0.j]
 [0.35355339+0.j]
 [0.35355339+0.j]
 [0.35355339+0.j]
 [0.35355339+0.j]]
| 010 >
